In [2]:
from torchvision import models
import torch

import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt


resnet18_pretrained = models.resnet18(pretrained=True)

In [3]:
num_classes= 18
num_ftrs = resnet18_pretrained.fc.in_features
resnet18_pretrained.fc = nn.Linear(num_ftrs, num_classes)

In [4]:
import json
with open("train.json", "r") as f:
    json_data = json.load(f)
json_data = json_data['train']

In [5]:
class TrainDataset(Dataset):
    def __init__(self, img_paths, transform, json_data):
        self.img_paths = img_paths
        self.transform = transform
        self.json_data = json_data
        
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        label = json_data[index//7]['img'][index%7]['class']
        
        return image, label

    def __len__(self):
        return len(self.img_paths)

In [6]:
import numpy as np
import torch

# 중앙을 중심으로 지킬앤 하이드 처럼 좌우에 컷믹스
def rand_bbox(size, lam):
    H = size[2]
    W = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)


    cx = np.random.randn() + W//2
    cy = np.random.randn() + H//2

    # 패치의 4점
    bbx1 = np.clip(cx - cut_w // 2, 0, W//2)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W//2)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return int(bbx1), int(bby1), int(bbx2), int(bby2)

In [16]:
# meta 데이터와 이미지 경로를 불러옵니다.
path_dir = 'input/data/train/images'

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = []
for image in json_data:
    for img in image['img']:
        image_paths.append(img['path'])
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    transforms.RandomHorizontalFlip(0.5),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TrainDataset(image_paths, transform, json_data)

loader = DataLoader(
    dataset,
    batch_size=64, # 
    shuffle=True # shuffle 추가
)
from sklearn.metrics import f1_score
device = torch.device('cuda')
model = resnet18_pretrained.to(device)
model.train()

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

for epoch in range(10):
    running_loss = 0.0
    epoch_f1 = 0
    n_iter = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # t_ = torch.zeros((100,18))
        # t_[range(100),labels]=1
        # print(t_)
        # t_ = t_.to(device)
        # 일정 확률로 컷믹스

        optimizer.zero_grad()

        if np.random.random() > 0.5 and (labels == 1 or labels == 4 or labels == 7 or labels == 10 or labels == 13 or labels == 16):

            rand_index = torch.randperm(inputs.size()[0]) # 패치에 사용할 label
            target_a = labels # 원본 이미지 label
            target_b = labels[rand_index] # 패치 이미지 label  
            lam = np.random.beta(1.0, 1.0)     
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            print(inputs.shape, bbx1, bby1, bbx2, bby2)

                # 원본 데이터에 컷믹스 패치
            inputs[:, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

                # 원본 이미지와 패치 이미지의 넓이 비율
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (inputs.size()[-1] * inputs.size()[-2]))

                # 예측은 레이블 1개
            outputs = model(inputs)

                # 원본 이미지의 레이블과 패치 이미지의 레이블에 대해 loss 가중합
            loss = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            
            # 컷믹스 안하면 그냥 별거 없음
        else:
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:
            print('{},{:.5f} loss: {:.3f}'.format(epoch+1, i+1, running_loss/10))
            running_loss=0.0
        n_iter += 1
    print('Finished')
    print('{:.4f}'.format(epoch_f1))

torch.save(model.state_dict(), '/opt/ml/resnet_cutmix.pt')

torch.Size([64, 3, 512, 384]) 45 60 192 452
torch.Size([64, 3, 512, 384]) 34 45 192 467
torch.Size([64, 3, 512, 384]) 95 127 192 383
torch.Size([64, 3, 512, 384]) 19 26 192 484
torch.Size([64, 3, 512, 384]) 80 105 192 403
1,10.00000 loss: 0.657
torch.Size([64, 3, 512, 384]) 12 15 192 495
torch.Size([64, 3, 512, 384]) 33 44 192 468
torch.Size([64, 3, 512, 384]) 77 104 192 410
torch.Size([64, 3, 512, 384]) 32 41 192 469
torch.Size([64, 3, 512, 384]) 34 46 192 466
torch.Size([64, 3, 512, 384]) 108 146 192 368
1,20.00000 loss: 0.560
torch.Size([64, 3, 512, 384]) 95 126 192 384
torch.Size([64, 3, 512, 384]) 105 141 192 373
torch.Size([64, 3, 512, 384]) 77 103 192 409
torch.Size([64, 3, 512, 384]) 56 73 192 441
1,30.00000 loss: 0.578
torch.Size([64, 3, 512, 384]) 1 4 192 508
torch.Size([64, 3, 512, 384]) 12 15 192 497
torch.Size([64, 3, 512, 384]) 54 73 192 435
torch.Size([64, 3, 512, 384]) 48 61 192 449
1,40.00000 loss: 0.472
torch.Size([64, 3, 512, 384]) 79 104 192 408
torch.Size([64, 3, 5